#author: Jedrzej Chmiel

In [1]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
!pip install nltk==3.7

In [6]:
import getpass
import os
github_username = '12jerek34jeremi'
github_password = getpass.getpass()
os.environ['GITHUB_AUTH'] = github_username+':'+github_password

In [7]:
!pip install git+https://$GITHUB_AUTH@github.com/12jerek34jeremi/harry_potter.git#subdirectory=hpcw

  Cloning https://****@github.com/12jerek34jeremi/harry_potter.git to c:\users\jedre\appdata\local\temp\pip-req-build-z3487fia


  Running command git clone --filter=blob:none --quiet 'https://****@github.com/12jerek34jeremi/harry_potter.git' 'C:\Users\jedre\AppData\Local\Temp\pip-req-build-z3487fia'
  fatal: unable to access 'https://github.com/12jerek34jeremi/harry_potter.git/': Failed to connect to github.com port 443 after 21144 ms: Timed out
  error: subprocess-exited-with-error
  
  git clone --filter=blob:none --quiet 'https://****@github.com/12jerek34jeremi/harry_potter.git' 'C:\Users\jedre\AppData\Local\Temp\pip-req-build-z3487fia' did not run successfully.
  exit code: 128
  
  See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

git clone --filter=blob:none --quiet 'https://****@github.com/12jerek34jeremi/harry_potter.git' 'C:\Users\jedre\AppData\Local\Temp\pip-req-build-z3487fia' did not run successfully.
exit code: 128

See above for output.

note: This error originates from a subprocess, and is likely no

In [ ]:
!python --version

In [1]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from hpcw.datasets.words_batch_dataset import WordsBatchDataset
from hpcw.datasets.one_item_dataset import OneItemDataset
from hpcw.corpus import Corpus
from hpcw.models.embedding import Embedding
from hpcw.models.words_batch import WordsBatch
from tqdm import tqdm
import os
import os.path
import pickle
from typing import List, Dict, Tuple
from datetime import datetime
from hpcw.utils import count_distance
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import nltk
nltk.download('punkt')

In [8]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [9]:
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
MAIN_DATA_DIR = 'drive/MyDrive/data_harry_potter'
MAIN_TRAINING_DIR = 'drive/MyDrive/harry_potter_model_training'
# MAIN_DATA_DIR = 'data'

In [11]:
corpus = Corpus(MAIN_DATA_DIR+'/dictionary/second_dictionary_30_04.pickle')
one_item_dataset = OneItemDataset(len(corpus))

In [12]:
# words_batch_datasets = [WordsBatchDataset(MAIN_DATA_DIR+'/harry_potter_books/prepared_txt/harry_potter_'+str(i)+'_prepared.txt', corpus.dictionary, sequence_length=6)
#                         for i in range(1,8)]

In [13]:
def test_words_batch(model:WordsBatch, datasets: List[OneItemDataset], batch_size=2048) -> float:
    """
Tests passed word batch model using all datasets and returns Mean Squared Error.
Parameters:
    model:
        Object of WordsBatch class to be tested.
    datasets:
        List of objects of WordsBatchDataset class. Datasets on which model will be tested.
    batch_size:
        The batch size to use.
Return:
    MSE (Mean Squared Error)
    """
    mse = 0.0
    with torch.no_grad():
      loss_function = nn.MSELoss(reduction='sum')
      for dataset in datasets:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)
        for X, y in loader:
          X = X.to(DEVICE)
          y = y.to(DEVICE)
          y = model.embedding.to_dense(y)
          pred = model(X)
          loss = loss_function(pred, y)
          mse +=  loss.item()
    
    total_length = sum([len(dataset) for dataset in datasets])
    mse = mse / total_length
    return mse

In [14]:
def train_words_batch(model:WordsBatch, datasets: List[OneItemDataset], batch_size: int, epochs: int,
                    optimizer: optim.Optimizer, saves_dir:str = None, results:Dict[str, int or float] = None,
                    start_epoch:int = 0):
    """
This function trains words_batch model. It uses MSE loss function to do this.
Parameters:
    model:
        Model to be trained. Object of WordsBatch class.
    datasets:
        List of all datasets on which model will be trained. First function uses all inputs from first dataset,
        then all input from second dataset, ..., then all outputs from last dataset. This is the end of first epoch.
    batch_size: the batch size to used
    epochs: How many epochs to do.
    optimizer: optimizer to use to update weights and biases.
    saves_dir: If not None in this directory function will save the model after each epoch. Files will be named as
        words_batch_epoch_1.pth, words_batch_epoch_2.pth, words_batch_epoch_3.pth, ...
        If in passed directory already exist file called for example words_batch_epoch_1.pth it will be truncated.
        If passed directory does not exist, it will be created.
    results:
        If saves_dir is None, then this argument is ignore.
        If saves_dir is not none and results is non None, then after each epoch model is tested using test_words_batch
        function and saves results in 'results' dictionary. Consequtives items of dictionary are named
        'mse_after_epoch_i', where i is prevoius epoch nr. After each epoch function saves 'results' dictionary using
        pickle module and as txt file. File are created in 'saves_dir' directory and named 'results.pickle', 'results.txt'.
    start_epoch:
        Number of first epoch. Important when funcction also tests models (if saves_dir and results are not None). If
        start_epoch is 8 and epochs_is 5, then function will train model over epochs 8,9,10,11 and 12.
    """
    model.train()
    loss_function = nn.MSELoss()
    loaders = [DataLoader(dataset, batch_size, shuffle=True) for dataset in datasets]
    if saves_dir is not None:
        if not os.path.exists(saves_dir):
            os.makedirs(saves_dir)
        with open(saves_dir+'/results.pickle', 'wb') as file:
                    pickle.dump(results, file)
        with open(saves_dir+'/results.txt', 'wt') as file:
            for key, item in results.items():
                file.write(key + ': ' + str(item) + '\n')

    end_epoch = start_epoch+epochs
    for epoch in range(start_epoch, end_epoch):
        for i,loader in enumerate(loaders):
            print(f"Epoch {epoch}/{end_epoch}, dataset {i+1}/7")
            for X, y in tqdm(loader, desc="batch: "):
                X = X.to(DEVICE)
                with torch.no_grad():
                    y = model.embedding.to_dense(y.to(DEVICE))
                pred = model(X)
                loss = loss_function(pred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        if saves_dir is not None:
            model.save(saves_dir+'/'+f"words_batch_epoch_{epoch}.pth")
            if results is not None:
                mse = test_words_batch(model, datasets)
                print(f"MSE is: {mse}")
                results[f'mse_after_epoch_{epoch}'] = mse
                with open(saves_dir+'/results.pickle', 'wb') as file:
                    pickle.dump(results, file)
                with open(saves_dir+'/results.txt', 'at') as file:
                    file.write(f'mse_after_epoch_{epoch} : {mse}\n')
                model.train()

In [16]:
def test_encoding(model: Embedding, dataset: OneItemDataset, batch_size=1024) -> :
    """
Tests passed embedding model using all datasets and returns accuracy factor.
(Accuracy factor is number of correctly classified samples devided by number of all samples.)
Parameters:
    model:
        Object of Embedding class to be tested.
    datasets:
        Object of OneItemDataset class. Data on which to test.
    batch_size:
        Size of batch to use.

Return:
    Accuracy factor (float).
    (Accuracy factor is number of correctly classified samples devided by number of all samples.)
    """
    acurate = 0
    model.eval()
    with torch.no_grad():
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)
        for y in loader:
            y = y.to(DEVICE)
            X = model.to_dense(y)
            pred = model(X)
            acurate += torch.count_nonzero(torch.argmin(pred,dim=1)==y).item()
    result = acurate/len(dataset)
    return  result

In [15]:
def train_encoding(model: Embedding, dataset: OneItemDataset,  batch_size: int, epochs: int,
                    optimizer: optim.Optimizer, saves_dir:str = None, results:Dict[str, int or float] = None,
                    start_epoch:int = 0, checkpoint_frequency:int = 1):
    """
This function trains encoding part of Embedding class object. It uses CrossEntropyLoss.
Parameters:
    model:
        Model to be trained. Object of Embedding class.
    datasets:
        Object of OneItemDataset class. Data on which to train.
    batch_size: The batch size to used.
    epochs: How many epochs to do.
    optimizer: optimizer to use to update weights and biases.
    saves_dir: If not None in this directory function will save the model after each epoch. Files will be named as
        words_batch_epoch_1.pth, words_batch_epoch_2.pth, words_batch_epoch_3.pth, ...
        If in passed directory already exist file called for example words_batch_epoch_1.pth it will be truncated.
        If passed directory does not exist, it will be created.
    results:
        If saves_dir is None, then this argument is ignore.
        If saves_dir is not none and results is non None, then after each epoch model is tested using test_encoding
        function and result is saved in 'results' dictionary. Consequtives items of dictionary are named
        'accuracy_after_epoch_i', where i is prevoius epoch nr. After each epoch function saves 'results' dictionary using
        pickle module and as txt file. File are created in 'saves_dir' directory and named 'results.pickle', 'results.txt'.
    start_epoch:
        Number of first epoch. Important when function also tests models (if saves_dir and results are not None). If
        start_epoch is 8 and epochs_is 5, then function will train model over epochs 8,9,10,11 and 12.
    frequency:
        If saves_dir is None, then this argument is ignored.
        After how many epochs to save and (or) test model. If frequency is 3, then model is saved after third epoch
        of training, after sixth epoch of training and so on... Frequency of saving does not depend on start_epoch.
        (It depends on number of epochs done by function, not by current_epoch number.)
Returns:
    List of acurate factor before each epoch.
    """

    model.train()
    loss_function = nn.CrossEntropyLoss()
    acurate = 0
    acurates = []
    loader = DataLoader(dataset, batch_size, shuffle=True)

    if saves_dir is not None:
            if not os.path.exists(saves_dir):
                os.makedirs(saves_dir)
            with open(saves_dir+'/results.pickle', 'wb') as file:
                        pickle.dump(results, file)
            with open(saves_dir+'/results.txt', 'wt') as file:
                for key, item in results.items():
                    file.write(key + ': ' + str(item) + '\n')

    bar = tqdm(range(start_epoch, start_epoch+epochs), desc='Epoch')
    for epoch in bar:
        for y in loader:
            y = y.to(DEVICE)
            with torch.no_grad():
                X = model.to_dense(y)
            pred = model(X)
            loss = loss_function(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if saves_dir is not None and (epoch-start_epoch) % checkpoint_frequency == 0:
            model.save_encoding(saves_dir+'/'+f"encoding_epoch_{epoch}.pth")
            if results is not None:
                acurate = test_encoding(model, dataset)
                bar.set_description(f"Acurate is: {acurate:.15f}, epoch: ")
                results[f'acurate_after_epoch_{epoch}'] = acurate
                with open(saves_dir+'/results.pickle', 'wb') as file:
                    pickle.dump(results, file)
                with open(saves_dir+'/results.txt', 'at') as file:
                    file.write(f'acurate_after_epoch_{epoch} : {acurate:.30f}\n')
                model.train()

In [17]:
# dir_name = MAIN_DATA_DIR+'/models/'+ datetime.now().strftime("training_words_batch_%d_%m_%Y___%H_%M")
# dir_name = MAIN_DATA_DIR+'/models/training_words_batch_11_05_2022___08_59'

In [18]:
# embedding = Embedding(corpus_size=len(corpus), embedding_size=64, dropout_factor=0.18, sizes=[128, 512])
# embedding = embedding.to(DEVICE)
# words_batch = WordsBatch(embedding, hidden_state_size=96, dropout_factor=0.18, sequence_length=6, dense_layer_size=256)
# words_batch = words_batch.to(DEVICE)

In [19]:
# results = words_batch.info()
# results['mse_initial'] = test_words_batch(words_batch, words_batch_datasets)

In [20]:
# words_batch = WordsBatch.load('/content/drive/MyDrive/data_harry_potter/models/training_words_batch_12_05_2022___09_22/words_batch_epoch_50.pth')
# words_batch = words_batch.to(DEVICE)
# with open(r'/content/drive/MyDrive/data_harry_potter/models/training_words_batch_12_05_2022___09_22/results.pickle', 'rb') as file:
#   results = pickle.load(file)

In [21]:
# for i in range(51, 64):
#   results.pop(f'mse_after_epoch_{i}')

In [22]:
# optimizer_words_batch = optim.Adam(words_batch.parameters())
# # results['optimizer_type'] = 'Adam'

In [23]:
# optimizer_words_batch = optim.SGD(words_batch.parameters(), lr=0.5)

In [24]:
# train_words_batch(words_batch, words_batch_datasets, batch_size=16, epochs=10,
#                   results=results, start_epoch=51, optimizer=optimizer_words_batch,
#                   saves_dir=dir_name)

In [25]:
# embedding_initial = Embedding(corpus_size=len(corpus), embedding_size=64, dropout_factor=0.18, sizes=[128, 512])
# words_batch0 = WordsBatch.load('/content/drive/MyDrive/data_harry_potter/models/training_words_batch_11_05_2022___08_59/words_batch_epoch_0.pth')
# words_batch34 = WordsBatch.load('/content/drive/MyDrive/data_harry_potter/models/training_words_batch_11_05_2022___08_59/words_batch_epoch_34.pth')
# words_batch61 = WordsBatch.load('/content/drive/MyDrive/data_harry_potter/models/training_words_batch_13_05_2022___09_19/words_batch_epoch_61.pth')
# words_batch93 = WordsBatch.load('/content/drive/MyDrive/data_harry_potter/models/training_words_batch_13_05_2022___13_29/words_batch_epoch_93.pth')

In [26]:
def check_distances(words: List[Tuple[str, str]], corpus: Corpus, embedding:Embedding):
  for word1, word2 in words:  
    dist = count_distance(word1, word2, corpus, embedding)
    dist **= 2
    print(f"    Distance between words '{word1}' and '{word2}' is {dist}")

In [27]:
words = [('cat', 'dog'), ('cat', 'going'), ('hermione', 'water'), ('hermione', 'harry'),
         ('is', 'being'), ('is', 'was'), ('hagrid', 'dumbledore'), ('snake', 'snakes'),
         ('book', 'brilliant'), ('food', 'voldemort'), ('girl', 'boy'), ('girls', 'boys')]

In [28]:
# print("For initial words_batch: ")
# check_distances(words, corpus, embedding_initial)

In [29]:
# print("For words_batch0: ")
# check_distances(words, corpus, words_batch0.embedding)

In [30]:
# print("For words_batch34: ")
# check_distances(words, corpus, words_batch34.embedding)

In [31]:
# print("For words_batch61: ")
# check_distances(words, corpus, words_batch61.embedding)

In [32]:
# print("For words_batch93: ")
# check_distances(words, corpus, words_batch93.embedding)

In [88]:
embedding = Embedding(corpus_size = len(corpus), embedding_size = 64, sizes=[256, 512, 1024], dropout_factor=0.18)
embedding = embedding.to(DEVICE)

In [89]:
embedding.load_embedding(MAIN_DATA_DIR + '/models/embedding_embedding93.pth')

True

In [90]:
dir_name = MAIN_DATA_DIR+'/models/'+ datetime.now().strftime("training_encoding_%d_%m_%Y___%H_%M")
dir_name

'drive/MyDrive/data_harry_potter/models/training_encoding_16_05_2022___13_34'

In [91]:
results = embedding.info()
results

{'corpus_size': 21371,
 'dropout_factor': 0.18,
 'embedding_size': 64,
 'sizes': [256, 512, 1024]}

In [92]:
results["initial_acurate"] = test_encoding(embedding, one_item_dataset)
results

{'corpus_size': 21371,
 'dropout_factor': 0.18,
 'embedding_size': 64,
 'initial_acurate': 4.679238220017781e-05,
 'sizes': [256, 512, 1024]}

In [99]:
# encoding_optimizer = optim.Adam(embedding.parameters(), lr=0.000001)
encoding_optimizer = optim.SGD(embedding.parameters(), lr=0.006)


In [ ]:
train_encoding(embedding, one_item_dataset, batch_size=16, epochs=10,
               optimizer=encoding_optimizer, saves_dir=dir_name, results = results,
               start_epoch=0, checkpoint_frequency=5)

Acurate is: 0.000000000000000, epoch: :  70%|███████   | 7/10 [00:43<00:18,  6.19s/it]